<a href="https://colab.research.google.com/github/lari-ms/IC/blob/main/lime%2Bsam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando dependencias da biblioteca segment_anything (sam)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/IC/xai')  # Ajuste o caminho se necessário


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install lime

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-oqz2jq6z
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-oqz2jq6z
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done


Importando bibliotecas

In [ ]:
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import sam_fns

import utils_fns as utils #module with aux functions
import sam_fns as samf #module with sam functions
import lime_fns as limef #module with lime functions

setando o ambiente

In [ ]:
using_colab = True

Iniciando explicabilidade

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-qn4i_h5p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-qn4i_h5p
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
mkdir: cannot create directory ‘images’: File exists
--2025-03-12 19:41:01--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99846 (98K) [image/jpeg]
Saving to: ‘images/dog.jpg.2’

dog

In [ ]:
#inicializando o sam
model_type = 'vit_h'
sam_checkpoint = 'sam_vit_h_4b8939.pth' #path to sam's weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #setando o ambiente?
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint) #fazendo o upload do modelo sam?
sam.to(device=device) #setando o dispositivo em q vai ser rodado?
mask_generator = SamAutomaticMaskGenerator(sam) #instancia um gerador automatico de mascara e passa o modelo sam pra ele

In [ ]:
#loading model
model = load_model('/content/drive/MyDrive/IC/xai/cat_dog_efficientnetb3.keras')
img_path = '/content/drive/MyDrive/IC/xai/gato.jpg'
img = utils.load_img(img_path, (250, 250))
img_for_prediction = tf.expand_dims(img/255.0, axis=0) #resizing and bathcing img

explained_img = limef.get_explanation(img_path, model, mask_generator)
